# Collect Central catalog end points

In [2]:
# Import libraries 
import urllib2
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from IPython.display import display
from collections import OrderedDict

print 'done'

done


# Prepare the output table 

In [3]:

CATALOG_SUMMARY_MV_merge_keys=['CountFIPS','DataCategory','DataType','HUC','OrganizationIdentifier','ReportIdentifier',
'ReportUnitIdentifier','ReportUnitName','State','WaDEURLAddress']

print CATALOG_SUMMARY_MV_merge_keys

['CountFIPS', 'DataCategory', 'DataType', 'HUC', 'OrganizationIdentifier', 'ReportIdentifier', 'ReportUnitIdentifier', 'ReportUnitName', 'State', 'WaDEURLAddress']


In [4]:
# CentralCatalog="http://www.westernstateswater.org/Idaho/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?"


States={

"ADWR":'http://www.arizonawade.com/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=ADWR', # Arizona

    
"CA_DWR":'http://wade.sdsc.edu/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=CA-DWR', # California
    
    
"CODWR":"http://www.westernstateswater.org/Colorado/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=CODWR", #COLORADO

    
"IDWR":"http://www.westernstateswater.org/Idaho/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=IDWR",  #  IDAHO 

    
"KS_KDA": "http://wade.kda.ks.gov/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=KS-KDA", # KANSAS
    

"NeDNR": "http://www.westernstateswater.org/Nebraska/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NeDNR", #NEBRASKA
    
    
"NDWR":"http://wade.water.nv.gov/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NDWR", #NEVADA   
   
    
"NMOSE":"http://www.westernstateswater.org/NewMexico/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NMOSE", # NEW MEXICO
  
        
"OWRD":"http://data.owrb.ok.gov:8080/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=OWRD", #OKLAHOMA 
  

"OREGON_WRD":"http://wade.wrd.state.or.us/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=OREGON-WRD"  , #OREGON 
   

"DENR":"http://www.westernstateswater.org/SouthDakota/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=DENR", #SOUTH DAKOTA
    
       
"UTWRE":"http://www.water.utah.gov/DWRE/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=utwre",   # UTAH: water rights
    

"UTDWRT":"http://www.water.utah.gov/DWRT/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=UTDWRT", # UTAH: water resources

    
"TCEQ":"http://www.westernstateswater.org/TCEQ/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=TCEQ", #TEXAS
       
    
"WYWDC":"http://www.westernstateswater.org/Wyoming/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=WYWDC", #WYOMING
    
    
"WYSEO":"http://www.westernstateswater.org/WYSEO/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=WYSEO" #WYOMING
    
}

print 'done'


done


# Append the XMLs and their states into one big list to loop over later

**It takes 1-2 min to finish**

In [5]:
xml_data_all={}

for state, URL in States.items():
    print state    
    xml_data=requests.get(URL).content
    
    xml_data_all[state] = xml_data
    
print 'done'    
# print xml_data_all

NDWR
UTDWRT
WYWDC
CODWR
DENR
ADWR
IDWR
OREGON_WRD
UTWRE
OWRD
NMOSE
CA_DWR
WYSEO
KS_KDA
NeDNR
TCEQ
done


# Call the Centeral Catalog

In [11]:


Orgs=['ADWR','CA_DWR','CODWR','IDWR','KS_KDA','NeDNR','NDWR','NMOSE','OWRD','OREGON_WRD','DENR','UTWRE','UTDWRT','TCEQ','WYWDC','WYSEO']


xml_data_all_CenteralCatalog={}

for org in Orgs:
    CenteralCatalog_URL='http://www.westernstateswater.org/CentralCatalog/WADE/v0.2/GetCatalog/GetCentralCatalog.php?orgid='+org
    print org   
    xml_data=requests.get(CenteralCatalog_URL).content
    
    xml_data_all_CenteralCatalog[org] = xml_data
print 'done'

ADWR
CA_DWR
CODWR
IDWR
KS_KDA
NeDNR
NDWR
NMOSE
OWRD
OREGON_WRD
DENR
UTWRE
UTDWRT
TCEQ
WYWDC
WYSEO
done


# Parse the XML into Pandas data frames and save them into CSVs

In [44]:
class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

big_data_frame = []
addition_orgs={}
Removal_orgs={}
NoChange_orgs={}

for state in xml_data_all.keys():
#     print state
    for org in xml_data_all_CenteralCatalog.keys():
        if state==org:
#             print org

            xml_data = xml_data_all[state]
            xml_data_CenteralCatalog = xml_data_all_CenteralCatalog[org]

            xml2df = XML2DataFrame(xml_data)
            xml2df_CenteralCatalog = XML2DataFrame(xml_data_CenteralCatalog)

            xml_dataframe = xml2df.process_data()
            xml_CenteralCatalog = xml2df_CenteralCatalog.process_data()

            (row,col)= xml_dataframe.shape
            # display (xml_dataframe)
#             print state + " web-service " +" has " +str(row) + ' rows' + ' and '+  str(col) + ' columns' 
  
            (row_CenteralCatalog,col)= xml_CenteralCatalog.shape
            # display (xml_dataframe)
#             print org + " centeral catalog " +" has " +str(row_CenteralCatalog) + ' rows' + ' and '+  str(col) + ' columns' 
#             print '---- \n \n'

            if row>row_CenteralCatalog:
                addition=row-row_CenteralCatalog
                addition_orgs[org]=addition
                
                print org+' has '+str(addition)+' new catalog enteries'
            
            elif row_CenteralCatalog<row:
                removals=row_CenteralCatalog-row
                Removal_orgs[org]=removals

                print org+' removed '+str(removals)+' from the the centeral catalog'

            else:
                print org+'has no change or updates compared to the centeral catalog'
                NoChange_orgs={org}
                
# print 'addition_orgs'
# print addition_orgs

# print Removal_orgs 

# print NoChange_orgs
            #     print row,col

#             xml_dataframe.columns = xml_dataframe.columns.str.replace("{http://www.exchangenetwork.net/schema/WaDE/0.2}", '')
#             x=xml_dataframe.keys()

        #     if not big_data_frame.empty:
        #         print big_data_frame
        #         big_data_frame.append(xml_dataframe)
        #     else: 
        #         print xml_dataframe
        #     big_data_frame.append(xml_dataframe)

        #     total_frame = pd.concat(xml_dataframe)
#             name=key+'.csv'
#             xml_dataframe.to_csv(name, index = False)
        #     print 'done'

NDWR has 31 new catalog enteries
WYWDChas no change or updates compared to the centeral catalog
CODWRhas no change or updates compared to the centeral catalog
DENRhas no change or updates compared to the centeral catalog
TCEQhas no change or updates compared to the centeral catalog
ADWRhas no change or updates compared to the centeral catalog
IDWRhas no change or updates compared to the centeral catalog
OREGON_WRD has 913 new catalog enteries
UTWRE has 4860 new catalog enteries
OWRD has 5918 new catalog enteries
NMOSEhas no change or updates compared to the centeral catalog
CA_DWR has 1021 new catalog enteries
WYSEO has 12 new catalog enteries
KS_KDA has 210 new catalog enteries
UTDWRT has 9 new catalog enteries
NeDNRhas no change or updates compared to the centeral catalog
